In [1]:
from hub_o import *
import pandas as pd
import folium
import os
import zipfile

# API POISSONS

### On commence par rechercher les stations de la Seine en filtrant par le libelle de la station

In [2]:
# Instancier l'API
api = EtatPiscicoleAPI()

# Requête avec filtre sur le nom (libellé) de la station
stations_seine = api.fetch_all_data("etat_piscicole/stations", params={"libelle_station": "seine"})

# print(stations_seine.columns)
print(stations_seine.shape)

⏱️ fetch_all_data exécutée en 0.27 s
(66, 68)


In [3]:
# Récupérer les stations
libelles_station = stations_seine["libelle_station"].tolist()
codes_station = stations_seine["code_station"].tolist()
print(libelles_station)

['LA SEINE A DUCLAIR 1', 'LA SEINE A VAL-DE-LA-HAYE 1', 'LA SEINE A ROUEN 2', 'LA SEINE A SAINT-AUBIN-LES-ELBEUF 1', 'LA SEINE A POSES 1', 'LA SEINE A POSES 1', 'LA SEINE A POSES 2', 'LA SEINE A SAINT-PIERRE-LA-GARENNE 1', 'LA SEINE A VETHEUIL 1', 'LA SEINE A MERICOURT 4', 'LA SEINE A TRIEL-SUR-SEINE 1', 'LA SEINE A POISSY 1', 'LA SEINE A ACHERES 1', 'LA SEINE A HERBLAY 1', 'LA SEINE A MAISONS-LAFFITTE 1', 'LA SEINE A CHATOU 3', 'LA SEINE A NANTERRE 1', 'LA SEINE A ARGENTEUIL 2', 'LA SEINE A ARGENTEUIL 1', 'LA SEINE A SURESNES 2', 'LA SEINE A SURESNES 2', 'LA SEINE A ISSY-LES-MOULINEAUX 1', 'LA SEINE A PARIS-15E--ARRONDISSEMENT 1', 'LA SEINE A ALFORTVILLE 2', 'LA SEINE A ABLON-SUR-SEINE 2', 'LA SEINE A SEINE-PORT 1', 'LA SEINE A SAINT-FARGEAU-PONTHIERRY 1', 'LA SEINE A AVON 1', 'LA SEINE A LA GRANDE-PAROISSE 1', 'LA SEINE A VARENNES-SUR-SEINE 1', 'LA SEINE A MONTEREAU-FAULT-YONNE 2', 'LA SEINE A MONTEREAU-FAULT-YONNE 1', 'LA SEINE A MAROLLES-SUR-SEINE 2', 'LA VIEILLE SEINE A MAROLLES-S

### Une fois qu'on a les stations de la Seine (libelle_station et code_station), on peut filtrer les différentes tables, on commence par les opération, puis on affiche sur une carte les stations en utilisant un code couleur pour voir quelles stations ont le plus d'opérations


In [4]:
# Supposons que tu as déjà la liste des codes station pour la Seine
codes_station = stations_seine["code_station"].tolist()

# Récupérer les opérations en filtrant sur ces codes_station
operations_seine = api.fetch_all_data("etat_piscicole/operations", params={"code_station": codes_station})

# Vérifier le résultat
print(operations_seine.columns)

⏱️ fetch_all_data exécutée en 0.78 s
Index(['code_operation', 'date_operation', 'etat_avancement_operation',
       'code_qualification_operation', 'libelle_qualification_operation',
       'code_station', 'libelle_station', 'uri_station',
       'coordonnee_x_station', 'coordonnee_y_station',
       ...
       'autres_especes_codes_taxon', 'autres_especes_codes_alternatifs_taxon',
       'autres_especes_noms_communs_taxon', 'autres_especes_noms_latins_taxon',
       'commentaire', 'libelle_point_prelevement_wama',
       'code_point_prelevement_wama', 'geometry', 'longitude', 'latitude'],
      dtype='object', length=186)


In [5]:
#On compte le nombre d'opérations par stations
ops_count = operations_seine.groupby("code_station")["code_operation"].nunique().reset_index(name="nombre_operations")
#print(ops_count)

In [6]:
stations_ops = stations_seine.merge(ops_count, on="code_station", how="left")
stations_ops["nombre_operations"] = stations_ops["nombre_operations"].fillna(0).astype(int)


In [7]:
import folium
import pandas as pd
import numpy as np

# Supposons que tu as déjà ton dataframe fusionné `stations_ops` avec au moins :
# 'latitude', 'longitude', 'nombre_operations', 'libelle_station'

# Crée une carte centrée sur la Seine (exemple : latitude 49.0, longitude 1.5)
m = folium.Map(location=[49.0, 1.5], zoom_start=8)

# Normalisation du nombre d'opérations pour la taille des cercles
max_ops = stations_ops["nombre_operations"].max()
min_ops = stations_ops["nombre_operations"].min()

def normalize_size(x, min_size=5, max_size=20):
    # Normalise le nombre d'opérations entre min_size et max_size
    if max_ops == min_ops:
        return (min_size + max_size) / 2
    return min_size + (x - min_ops) / (max_ops - min_ops) * (max_size - min_size)

# Fonction pour la couleur (du vert au rouge)
def color_scale(x):
    # On normalise aussi entre 0 et 1
    norm = (x - min_ops) / (max_ops - min_ops) if max_ops != min_ops else 0.5
    # Plus le nombre est grand, plus la couleur tire vers le rouge
    r = int(255 * norm)
    g = int(255 * (1 - norm))
    b = 0
    return f"#{r:02x}{g:02x}{b:02x}"

# Ajout des cercles
for _, row in stations_ops.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=normalize_size(row["nombre_operations"]),
        color=color_scale(row["nombre_operations"]),
        fill=True,
        fill_color=color_scale(row["nombre_operations"]),
        fill_opacity=0.7,
        popup=f"{row['libelle_station']}: {row['nombre_operations']} opérations"
    ).add_to(m)

# Affiche la carte
m



### On fait de même pour les observations

In [8]:
# Récupérer les opérations en filtrant sur ces codes_station
observations_seine = api.fetch_all_data("etat_piscicole/observations", params={"code_station": codes_station})

# Vérifier le résultat
print(observations_seine.shape)

⏱️ fetch_all_data exécutée en 84.10 s
(94942, 114)


In [9]:
# Supposons que tu as déjà récupéré les données stations, opérations et observations
# stations_seine, operations_seine, observations_seine

# Comptage du nombre d'opérations uniques par station
ops_count = operations_seine.groupby("code_station")["code_operation"].nunique().reset_index(name="nombre_operations")

# Comptage du nombre total d'observations par station (toutes observations confondues)
obs_count = observations_seine.groupby("code_station").size().reset_index(name="nombre_observations")

# Fusion avec les stations pour avoir les noms et infos utiles
stations_summary = stations_seine[["code_station", "libelle_station"]].merge(ops_count, on="code_station", how="left")
stations_summary = stations_summary.merge(obs_count, on="code_station", how="left")

# Remplacer les NaN par 0 si certaines stations n'ont pas d'opérations ou d'observations
stations_summary["nombre_operations"] = stations_summary["nombre_operations"].fillna(0).astype(int)
stations_summary["nombre_observations"] = stations_summary["nombre_observations"].fillna(0).astype(int)

# Afficher le tableau final
print(stations_summary[["code_station", "nombre_operations", "nombre_observations"]])


   code_station  nombre_operations  nombre_observations
0      03184370                  3                   32
1      03183758                 13                  834
2      03183556                  2                   62
3      03182669                  3                  123
4      03174021                 16                 1221
..          ...                ...                  ...
61     03000118                  3                  688
62     03000115                  2                  428
63     03000111                  3                  456
64     03000100                  6                 1420
65     03000084                  1                    9

[66 rows x 3 columns]


In [10]:
import folium
from folium import CircleMarker

# On ajoute les coordonnées aux données résumées en fusionnant avec stations_seine
stations_summary = stations_summary.merge(
    stations_seine[["code_station", "latitude", "longitude"]],
    on="code_station",
    how="left"
)

# Centrer la carte sur la moyenne des coordonnées
mean_lat = stations_summary["latitude"].mean()
mean_lon = stations_summary["longitude"].mean()

# Créer la carte
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=9)

# Définir les tailles de cercle en fonction du nombre d'opérations
min_ops = stations_summary["nombre_operations"].min()
max_ops = stations_summary["nombre_operations"].max()

def size_scale(ops):
    if max_ops == min_ops:
        return 10
    return 5 + 15 * (ops - min_ops) / (max_ops - min_ops)

# Définir la couleur en fonction du nombre d'observations (du vert au rouge)
min_obs = stations_summary["nombre_observations"].min()
max_obs = stations_summary["nombre_observations"].max()

def color_scale(obs):
    if max_obs == min_obs:
        return "green"
    ratio = (obs - min_obs) / (max_obs - min_obs)
    # gradient vert (peu d'observations) -> rouge (beaucoup d'observations)
    red = int(255 * ratio)
    green = int(255 * (1 - ratio))
    return f"#{red:02x}{green:02x}00"

# Ajouter les points sur la carte
for _, row in stations_summary.iterrows():
    size = size_scale(row["nombre_operations"])
    color = color_scale(row["nombre_observations"])
    popup_text = (
        f"<b>{row['libelle_station']}</b><br>"
        f"Opérations : {row['nombre_operations']}<br>"
        f"Observations : {row['nombre_observations']}"
    )
    CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=size,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=popup_text,
    ).add_to(m)

# Afficher la carte
m


# API QUALITE COURS D'EAU
### On va d'abord vérifier quelles sont les stations communes aux API poisson et qualité des cours d'eau

In [11]:
api_eau = QualiteCoursEauAPI()

# Requête avec filtre sur le nom (libellé) de la station
station_pc_seine = api_eau.fetch_all_data("qualite_rivieres/station_pc", params={"libelle_station": "seine"})

# print(stations_seine.columns)
print(station_pc_seine.shape)


⏱️ fetch_all_data exécutée en 0.94 s
(131, 44)


In [12]:
# Récupérer les stations
libelles_station_pc = station_pc_seine["libelle_station"].tolist()
codes_station_pc = station_pc_seine["code_station"].tolist()
print(libelles_station)

['LA SEINE A DUCLAIR 1', 'LA SEINE A VAL-DE-LA-HAYE 1', 'LA SEINE A ROUEN 2', 'LA SEINE A SAINT-AUBIN-LES-ELBEUF 1', 'LA SEINE A POSES 1', 'LA SEINE A POSES 1', 'LA SEINE A POSES 2', 'LA SEINE A SAINT-PIERRE-LA-GARENNE 1', 'LA SEINE A VETHEUIL 1', 'LA SEINE A MERICOURT 4', 'LA SEINE A TRIEL-SUR-SEINE 1', 'LA SEINE A POISSY 1', 'LA SEINE A ACHERES 1', 'LA SEINE A HERBLAY 1', 'LA SEINE A MAISONS-LAFFITTE 1', 'LA SEINE A CHATOU 3', 'LA SEINE A NANTERRE 1', 'LA SEINE A ARGENTEUIL 2', 'LA SEINE A ARGENTEUIL 1', 'LA SEINE A SURESNES 2', 'LA SEINE A SURESNES 2', 'LA SEINE A ISSY-LES-MOULINEAUX 1', 'LA SEINE A PARIS-15E--ARRONDISSEMENT 1', 'LA SEINE A ALFORTVILLE 2', 'LA SEINE A ABLON-SUR-SEINE 2', 'LA SEINE A SEINE-PORT 1', 'LA SEINE A SAINT-FARGEAU-PONTHIERRY 1', 'LA SEINE A AVON 1', 'LA SEINE A LA GRANDE-PAROISSE 1', 'LA SEINE A VARENNES-SUR-SEINE 1', 'LA SEINE A MONTEREAU-FAULT-YONNE 2', 'LA SEINE A MONTEREAU-FAULT-YONNE 1', 'LA SEINE A MAROLLES-SUR-SEINE 2', 'LA VIEILLE SEINE A MAROLLES-S

In [13]:
codes_station = set(stations_seine["code_station"])
codes_station_pc = set(station_pc_seine["code_station"])

# Codes en commun
communs = codes_station & codes_station_pc
print(f"Codes communs ({len(communs)}): {sorted(communs)}")

# Codes dans stations_seine mais pas dans stations_seine_pc
uniques_seine = codes_station - codes_station_pc
print(f"Codes uniquement dans stations_seine ({len(uniques_seine)}): {sorted(uniques_seine)}")

# Codes dans stations_seine_pc mais pas dans stations_seine
uniques_pc = codes_station_pc - codes_station
print(f"Codes uniquement dans stations_seine_pc ({len(uniques_pc)}): {sorted(uniques_pc)}")


Codes communs (19): ['03000100', '03001000', '03002100', '03002127', '03004349', '03006000', '03011300', '03012100', '03012800', '03014000', '03048000', '03063000', '03082000', '03084470', '03125000', '03125500', '03127370', '03174000', '03184370']
Codes uniquement dans stations_seine (41): ['03000084', '03000111', '03000115', '03000118', '03002040', '03002061', '03002107', '03002145', '03002174', '03002228', '03002265', '03011676', '03011938', '03012375', '03012389', '03012602', '03012612', '03012895', '03013590', '03013798', '03013854', '03023150', '03023401', '03023653', '03045884', '03048232', '03080339', '03081458', '03081647', '03083222', '03083250', '03083700', '03083855', '03084814', '03084880', '03127623', '03172471', '03174021', '03182669', '03183556', '03183758']
Codes uniquement dans stations_seine_pc (112): ['03000033', '03000180', '03000450', '03000456', '03000542', '03000560', '03001300', '03001490', '03002000', '03002021', '03002150', '03003000', '03003140', '03003290',

In [14]:
import folium

# Étape 1 : Identifier les codes en commun
codes_communs = set(stations_seine["code_station"]) & set(station_pc_seine["code_station"])

# Étape 2 : Filtrer les stations avec ces codes
stations_communes = stations_seine[stations_seine["code_station"].isin(codes_communs)]

# Étape 3 : Créer la carte centrée sur la moyenne des coordonnées
moyenne_lat = stations_communes["latitude"].mean()
moyenne_lon = stations_communes["longitude"].mean()
carte = folium.Map(location=[moyenne_lat, moyenne_lon], zoom_start=8)

# Ajouter les marqueurs
for _, row in stations_communes.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["libelle_station"],
        tooltip=row["code_station"]
    ).add_to(carte)

# Afficher la carte
carte


In [15]:
import folium

# Ta liste de stations utiles
libelle_station_utiles = [
    'LA SEINE A POSES 2',
    'LA SEINE A SURESNES 2',
    'LA SEINE A CONFLANS-SUR-SEINE 1',
    'LA SEINE A NOD-SUR-SEINE 1'
]

# Filtrer les deux DataFrames
stations_seine_utiles = stations_seine[stations_seine["libelle_station"].isin(libelle_station_utiles)]
stations_seine_pc_utiles = station_pc_seine[station_pc_seine["libelle_station"].isin(libelle_station_utiles)]

# Carte centrée sur la moyenne des lat/lon
lat_moy = pd.concat([stations_seine_utiles["latitude"], stations_seine_pc_utiles["latitude"]]).mean()
lon_moy = pd.concat([stations_seine_utiles["longitude"], stations_seine_pc_utiles["longitude"]]).mean()
m = folium.Map(location=[lat_moy, lon_moy], zoom_start=8)

# Marqueurs pour stations_seine (bleu)
for _, row in stations_seine_utiles.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=6,
        color='blue',
        fill=True,
        fill_opacity=0.6,
        popup=f"Etat piscicole : {row['libelle_station']}",
        tooltip=row['code_station']
    ).add_to(m)

# Marqueurs pour stations_seine_pc (rouge)
for _, row in stations_seine_pc_utiles.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=6,
        color='red',
        fill=True,
        fill_opacity=0.6,
        popup=f"Qualité rivières : {row['libelle_station']}",
        tooltip=row['code_station']
    ).add_to(m)

# Affichage ou sauvegarde
m  # ou m.save("comparaison_stations.html")


### On va récupérer les données des cours d'eau pour 4 stations intéressantes (on récupère aussi au passage les indicateurs de l'API poisson)

In [18]:
libelles_utiles = ['LA SEINE A POSES 2',
                   'LA SEINE A SURESNES 2',
                   'LA SEINE A CONFLANS-SUR-SEINE 1',
                   'LA SEINE A NOD-SUR-SEINE 1']

# Filtrer les stations selon les libellés
stations_filtrees = station_pc_seine[station_pc_seine['libelle_station'].isin(libelles_utiles)]

# Extraire les codes nécessaires pour les params
codes_station_pc = stations_filtrees['code_station'].unique().tolist()

# Construire les params à passer aux APIs
params_qualite = {"code_station": codes_station_pc}

indicateurs_seine = api.fetch_all_data("etat_piscicole/indicateurs", params={"code_station": codes_station})
qualite_eau_condition_pc = api_eau.fetch_all_data(api_eau.endpoint_condition_env_pc, params_qualite)
qualite_eau_operation_pc = api_eau.fetch_all_data(api_eau.endpoint_operation_pc, params_qualite)
qualite_eau_station_pc = api_eau.fetch_all_data(api_eau.endpoint_station_pc, params_qualite)
qualite_eau_analyse_pc = api_eau.fetch_all_data_by_year_range(api_eau.endpoint_analyse_pc, params_qualite, start_year=1997, end_year=2024)

⏱️ fetch_all_data exécutée en 1.45 s
⏱️ fetch_all_data exécutée en 5.28 s
⏱️ fetch_all_data exécutée en 5.51 s
⏱️ fetch_all_data exécutée en 0.49 s
Récupération des données du 1997-01-01 au 1997-12-31
⏱️ fetch_all_data exécutée en 2.99 s
📈 3695 lignes récupérées jusqu'à présent.
Récupération des données du 1998-01-01 au 1998-12-31
⏱️ fetch_all_data exécutée en 2.38 s
📈 7129 lignes récupérées jusqu'à présent.
Récupération des données du 1999-01-01 au 1999-12-31
⏱️ fetch_all_data exécutée en 2.27 s
📈 10962 lignes récupérées jusqu'à présent.
Récupération des données du 2000-01-01 au 2000-12-31
⏱️ fetch_all_data exécutée en 2.16 s
📈 14873 lignes récupérées jusqu'à présent.
Récupération des données du 2001-01-01 au 2001-12-31
⏱️ fetch_all_data exécutée en 2.43 s
📈 19066 lignes récupérées jusqu'à présent.
Récupération des données du 2002-01-01 au 2002-12-31
⏱️ fetch_all_data exécutée en 2.73 s
📈 23535 lignes récupérées jusqu'à présent.
Récupération des données du 2003-01-01 au 2003-12-31
⏱️ 

### On stock toutes les données dans un dictionnaire

In [19]:
dataset_seine = {"poissons_stations": stations_seine,
                "poissons_observations": observations_seine,
                "poissons_operations": operations_seine,
                "poissons_indicateurs": indicateurs_seine,
                "qualite_eau_station_pc": qualite_eau_station_pc,
                "qualite_eau_analyse_pc": qualite_eau_analyse_pc,
                "qualite_eau_condition_env_pc": qualite_eau_condition_pc,
                "qualite_eau_operation_pc": qualite_eau_operation_pc}
    

In [20]:
print("=== Tailles des DataFrames dans dataset_seine ===")
for key, df in dataset_seine.items():
    print(f"\nItem : '{key}'")
    if df is not None:
        print(f"Nombre de lignes : {df.shape[0]}")
        print(f"Nombre de colonnes : {df.shape[1]}")
    else:
        print("Aucun dataframe (valeur None)")
print("\n=== Fin des tailles ===")


=== Tailles des DataFrames dans dataset_seine ===

Item : 'poissons_stations'
Nombre de lignes : 66
Nombre de colonnes : 68

Item : 'poissons_observations'
Nombre de lignes : 94942
Nombre de colonnes : 114

Item : 'poissons_operations'
Nombre de lignes : 605
Nombre de colonnes : 186

Item : 'poissons_indicateurs'
Nombre de lignes : 373
Nombre de colonnes : 156

Item : 'qualite_eau_station_pc'
Nombre de lignes : 4
Nombre de colonnes : 44

Item : 'qualite_eau_analyse_pc'
Nombre de lignes : 400748
Nombre de colonnes : 72

Item : 'qualite_eau_condition_env_pc'
Nombre de lignes : 10932
Nombre de colonnes : 44

Item : 'qualite_eau_operation_pc'
Nombre de lignes : 3718
Nombre de colonnes : 44

=== Fin des tailles ===


### Sauvegarde des données dans un dossier dédié

In [21]:
# Dossier mère
base_dir = "Seine"
poissons_dir = os.path.join(base_dir, "Poissons")
qualite_dir = os.path.join(base_dir, "Qualite des Cours d'Eau")

# Création des dossiers si besoin
os.makedirs(poissons_dir, exist_ok=True)
os.makedirs(qualite_dir, exist_ok=True)

# Clés associées à chaque dossier
poissons_keys = [
    'poissons_stations',
    'poissons_observations',
    'poissons_operations',
    'poissons_indicateurs',
]

qualite_keys = [
    'qualite_eau_station_pc',
    'qualite_eau_analyse_pc',
    'qualite_eau_condition_env_pc',
    'qualite_eau_operation_pc',
]

# Sauvegarde des CSV
for key, df in dataset_seine.items():
    if key in poissons_keys:
        folder = poissons_dir
    elif key in qualite_keys:
        folder = qualite_dir
    else:
        # Si une clé ne correspond à aucun dossier, on l'ignore ou on la met ailleurs
        print(f"Avertissement : clé '{key}' non catégorisée, fichier non sauvegardé.")
        continue

    # Construction chemin complet fichier csv
    filepath = os.path.join(folder, f"{key}.csv")

    # Sauvegarde en csv
    df.to_csv(filepath, index=False)
    print(f"Enregistré : {filepath} (taille : {df.shape[0]} lignes)")

# Compression du dossier Seine en zip
zip_path = base_dir + ".zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(base_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, base_dir)
            zipf.write(file_path, arcname)
print(f"Dossier '{base_dir}' compressé en '{zip_path}'")


Enregistré : Seine\Poissons\poissons_stations.csv (taille : 66 lignes)
Enregistré : Seine\Poissons\poissons_observations.csv (taille : 94942 lignes)
Enregistré : Seine\Poissons\poissons_operations.csv (taille : 605 lignes)
Enregistré : Seine\Poissons\poissons_indicateurs.csv (taille : 373 lignes)
Enregistré : Seine\Qualite des Cours d'Eau\qualite_eau_station_pc.csv (taille : 4 lignes)
Enregistré : Seine\Qualite des Cours d'Eau\qualite_eau_analyse_pc.csv (taille : 400748 lignes)
Enregistré : Seine\Qualite des Cours d'Eau\qualite_eau_condition_env_pc.csv (taille : 10932 lignes)
Enregistré : Seine\Qualite des Cours d'Eau\qualite_eau_operation_pc.csv (taille : 3718 lignes)
Dossier 'Seine' compressé en 'Seine.zip'
